In [1]:
# 1-3.py
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# .env 파일 로드
load_dotenv()

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성 (이전 노트북에서 사용된 방식 그대로)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct", # 또는 다른 Groq 모델
    temperature=0.7
)

# 예시 데이터 (제공된 2개 + 1개 추가)
examples = [
    {
        "news": "삼성전자가 내년 중 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대처하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    { # 추가 예시
        "news": "한국은행은 물가 안정을 위해 기준금리를 현재 수준으로 유지하기로 결정했다. 이는 경기 둔화 우려와 소비자 물가 상승 압력을 동시에 고려한 조치로 풀이된다. 전문가들은 향후 경제 지표를 주시하며 추가적인 정책 변화 가능성을 열어두고 있다.",
        "keywords": "한국은행, 기준금리, 물가안정"
    }
]

# 예시 프롬프트 템플릿
# "human": "{news}", "ai": "키워드: {keywords}" 형식
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{news}"),
        ("ai", "키워드: {keywords}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
# 시스템 메시지 포함
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 뉴스 기사에서 핵심 키워드를 추출하는 전문가입니다. 주어진 뉴스에서 가장 중요한 키워드 3개를 쉼표로 구분하여 '키워드: 키워드1, 키워드2, 키워드3' 형식으로 추출해 주세요."),
        few_shot_prompt,
        ("human", "{input_news}"), # 사용자 입력 뉴스
    ]
)

# 체인 구성: final_prompt | llm | output_parser
output_parser = StrOutputParser()
keyword_extractor_chain = final_prompt | llm | output_parser

# 테스트 뉴스 (제공된 이미지의 "테스트 뉴스" 활용)
test_news_1 = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실힘 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

# 다양한 분야의 뉴스로 테스트
test_news_2 = "기후 변화는 전 세계적인 문제이며, 각국 정부는 탄소 배출량 감축 목표를 설정하고 재생 에너지 기술 개발에 투자하고 있습니다. 특히 젊은 세대들은 환경 문제에 대한 목소리를 높이며 적극적인 행동을 요구하고 있습니다."
test_news_3 = "최근 출시된 스마트폰 신제품은 혁신적인 카메라 기능과 더욱 강력해진 배터리 성능을 자랑한다. 디자인 또한 전작보다 훨씬 슬림해지고 세련된 느낌을 준다. 사용자들은 높은 만족도를 보이며 예약 판매량이 급증하고 있다."


print("--- 테스트 뉴스 1 ---")
print(f"뉴스: {test_news_1}")
try:
    result1 = keyword_extractor_chain.invoke({"input_news": test_news_1})
    print(result1)
except Exception as e:
    print(f"오류 발생: {e}")

print("\n--- 테스트 뉴스 2 ---")
print(f"뉴스: {test_news_2}")
try:
    result2 = keyword_extractor_chain.invoke({"input_news": test_news_2})
    print(result2)
except Exception as e:
    print(f"오류 발생: {e}")

print("\n--- 테스트 뉴스 3 ---")
print(f"뉴스: {test_news_3}")
try:
    result3 = keyword_extractor_chain.invoke({"input_news": test_news_3})
    print(result3)
except Exception as e:
    print(f"오류 발생: {e}")

--- 테스트 뉴스 1 ---
뉴스: 제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실힘 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.
키워드: 제미나이, 구글, AI모델

--- 테스트 뉴스 2 ---
뉴스: 기후 변화는 전 세계적인 문제이며, 각국 정부는 탄소 배출량 감축 목표를 설정하고 재생 에너지 기술 개발에 투자하고 있습니다. 특히 젊은 세대들은 환경 문제에 대한 목소리를 높이며 적극적인 행동을 요구하고 있습니다.
키워드: 기후변화, 탄소배출량, 재생에너지

--- 테스트 뉴스 3 ---
뉴스: 최근 출시된 스마트폰 신제품은 혁신적인 카메라 기능과 더욱 강력해진 배터리 성능을 자랑한다. 디자인 또한 전작보다 훨씬 슬림해지고 세련된 느낌을 준다. 사용자들은 높은 만족도를 보이며 예약 판매량이 급증하고 있다.
키워드: 스마트폰, 카메라, 배터리
